In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


DATA PATH

In [ ]:
# Load the data
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')


# fnn layers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

DATA PROCESSING

In [ ]:
# loading the DATA
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')

# Concatination features on the bases of 'name'
merged_data = acoustic_data.merge(paralinguistic_data, on='name').merge(linguistic_data, on='name')

# DROP the unwanted columns
X = merged_data.drop(columns=['name', 'Class']).values
y = merged_data['Class'].values

# Standardize and normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=65, random_state=42)


In [ ]:
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, num_layers, num_classes, ffnn_hidden_dim):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Feedforward layer
        self.ffnn = nn.Linear(hidden_dim, ffnn_hidden_dim)
        self.relu = nn.ReLU()

        # Final classification layer
        self.fc = nn.Linear(ffnn_hidden_dim, num_classes)

    def forward(self, src):
        src = self.embedding(src)
        out = self.transformer_encoder(src)

        # Applying the feedforward layer
        out = self.ffnn(out)
        out = self.relu(out)

        out = self.fc(out)
        return out


In [ ]:
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# Ensure this matches the number of features in your input data
input_dim = X_train.shape[1]  # Update this based on your actual input features

# hidden_dim
hidden_dim = 128
num_heads = 4
num_layers = 2
num_classes = len(set(y_train))  # TO Calculate the number of classes
ffnn_hidden_dim = 256  # adjust as of requirement

model = TransformerEncoder(input_dim=input_dim, num_heads=num_heads, hidden_dim=hidden_dim, num_layers=num_layers, num_classes=num_classes, ffnn_hidden_dim=ffnn_hidden_dim)

#Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Data Loader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}')


# Evaluate the model on the test set
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy on Test Data: {accuracy:.2f}%')

# Calculate and print the classification report
true_labels = y_test  # True labels from the test set
predicted_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.cpu().numpy())

classification_rep = classification_report(true_labels, predicted_labels)
print("Classification Report:\n", classification_rep)

Epoch 1/100, Average Loss: 0.2694
Epoch 2/100, Average Loss: 0.0318
Epoch 3/100, Average Loss: 0.0167
Epoch 4/100, Average Loss: 0.0948
Epoch 5/100, Average Loss: 0.0493
Epoch 6/100, Average Loss: 0.0243
Epoch 7/100, Average Loss: 0.0120
Epoch 8/100, Average Loss: 0.0027
Epoch 9/100, Average Loss: 0.0019
Epoch 10/100, Average Loss: 0.0016
Epoch 11/100, Average Loss: 0.0010
Epoch 12/100, Average Loss: 0.0006
Epoch 13/100, Average Loss: 0.0005
Epoch 14/100, Average Loss: 0.0004
Epoch 15/100, Average Loss: 0.0003
Epoch 16/100, Average Loss: 0.0003
Epoch 17/100, Average Loss: 0.0002
Epoch 18/100, Average Loss: 0.0002
Epoch 19/100, Average Loss: 0.0002
Epoch 20/100, Average Loss: 0.0007
Epoch 21/100, Average Loss: 0.0002
Epoch 22/100, Average Loss: 0.0003
Epoch 23/100, Average Loss: 0.0342
Epoch 24/100, Average Loss: 0.1461
Epoch 25/100, Average Loss: 0.2318
Epoch 26/100, Average Loss: 0.1560
Epoch 27/100, Average Loss: 0.0544
Epoch 28/100, Average Loss: 0.0769
Epoch 29/100, Average Loss: 0

In [ ]:
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
# Load the data
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')

for inputs, labels in train_loader:
    print("Batch input shape:", inputs.shape)
    print("Batch labels shape:", labels.shape)
    break

# Concatination datasets on 'name'
merged_data = acoustic_data.merge(paralinguistic_data, on='name').merge(linguistic_data, on='name')

# Preprocess data
X = merged_data.drop(columns=['name', 'Class']).values
y = merged_data['Class'].values

# Standardize and normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=45, random_state=42)

class TransformerEncoder(nn.Module):# Defining transformer encoder
    def __init__(self, input_dim, num_heads, hidden_dim, num_layers, num_classes):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Initialize the final linear layer with num_classes
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, src):
        src = self.embedding(src)
        out = self.transformer_encoder(src)
        # No averaging, directly pass to final layer
        out = self.fc(out)
        return out




# Ensure this matches the number of features in your input data
input_dim = X_train.shape[1]  # Update this based on your actual input features

# Update the hidden_dim if necessary
hidden_dim = 128  # This can be different but should align with your model architecture
num_heads = 4
num_layers = 2
num_classes = len(set(y_train))  # Calculate the number of classes

model = TransformerEncoder(input_dim=input_dim, num_heads=num_heads, hidden_dim=hidden_dim, num_layers=num_layers, num_classes=num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Data Loader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}')

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy on Test Data: {accuracy:.2f}%')


Batch input shape: torch.Size([32, 1800])
Batch labels shape: torch.Size([32])
Epoch 1/100, Average Loss: 0.2989
Epoch 2/100, Average Loss: 0.0263
Epoch 3/100, Average Loss: 0.0111
Epoch 4/100, Average Loss: 0.0034
Epoch 5/100, Average Loss: 0.0007
Epoch 6/100, Average Loss: 0.0004
Epoch 7/100, Average Loss: 0.1840
Epoch 8/100, Average Loss: 0.0641
Epoch 9/100, Average Loss: 0.1607
Epoch 10/100, Average Loss: 0.0494
Epoch 11/100, Average Loss: 0.0090
Epoch 12/100, Average Loss: 0.0133
Epoch 13/100, Average Loss: 0.0026
Epoch 14/100, Average Loss: 0.0022
Epoch 15/100, Average Loss: 0.0016
Epoch 16/100, Average Loss: 0.0014
Epoch 17/100, Average Loss: 0.0011
Epoch 18/100, Average Loss: 0.0008
Epoch 19/100, Average Loss: 0.0007
Epoch 20/100, Average Loss: 0.0008
Epoch 21/100, Average Loss: 0.0008
Epoch 22/100, Average Loss: 0.0005
Epoch 23/100, Average Loss: 0.0006
Epoch 24/100, Average Loss: 0.0004
Epoch 25/100, Average Loss: 0.0004
Epoch 26/100, Average Loss: 0.0004
Epoch 27/100, Averag

# try to apply of encoder

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
# Load the data
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')

for inputs, labels in train_loader:
    print("Batch input shape:", inputs.shape)
    print("Batch labels shape:", labels.shape)
    break
# Concatenation datasets on 'name'
merged_data = acoustic_data.merge(paralinguistic_data, on='name').merge(linguistic_data, on='name')

# Preprocess data
X = merged_data.drop(columns=['name', 'Class']).values
y = merged_data['Class'].values

# Standardize and normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=65, random_state=0)

# Indices for feature splits
acoustic_end = 768
paralinguistic_end = acoustic_end + 1024


class FeatureAttention(nn.Module):
    def __init__(self, feature_dim, attention_dim):
        super(FeatureAttention, self).__init__()
        self.attention_weight = nn.Linear(feature_dim, attention_dim)
        self.context_vector = nn.Linear(attention_dim, 1, bias=False)

    def forward(self, x):
        attn = torch.tanh(self.attention_weight(x))
        score = self.context_vector(attn)
        attention_weights = torch.softmax(score, dim=1)
        return torch.sum(x * attention_weights, dim=1)

class TransformerEncoder(nn.Module):
    def __init__(self, acoustic_dim, paralinguistic_dim, linguistic_dim, hidden_dim, num_classes, ffnn_hidden_dim):
        super(TransformerEncoder, self).__init__()
        self.acoustic_attention = FeatureAttention(acoustic_dim, hidden_dim)
        self.paralinguistic_attention = FeatureAttention(paralinguistic_dim, hidden_dim)
        self.linguistic_attention = FeatureAttention(linguistic_dim, hidden_dim)

        self.embedding = nn.Linear(hidden_dim * 3, hidden_dim)  # x3 for concatenation of feature attentions
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)

        self.ffnn = nn.Linear(hidden_dim, ffnn_hidden_dim)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(ffnn_hidden_dim, num_classes)

    def forward(self, acoustic_features, paralinguistic_features, linguistic_features):
        acoustic_context = self.acoustic_attention(acoustic_features)
        paralinguistic_context = self.paralinguistic_attention(paralinguistic_features)
        linguistic_context = self.linguistic_attention(linguistic_features)

        # Combine the contexts
        combined_context = torch.cat((acoustic_context, paralinguistic_context, linguistic_context), dim=1)
        combined_context = self.embedding(combined_context)
        combined_context = self.transformer_encoder(combined_context.unsqueeze(1))

        out = self.ffnn(combined_context.squeeze(1))
        out = self.relu(out)
        out = self.fc(out)
        return out

# Model parameters
acoustic_dim = 768  # Number of acoustic features
paralinguistic_dim = 1024  # Number of paralinguistic features
linguistic_dim = 6  # Number of linguistic features
hidden_dim = 128
num_heads = 4
num_layers = 2
num_classes = len(set(y_train))
ffnn_hidden_dim = 256


model = TransformerEncoder(acoustic_dim, paralinguistic_dim, linguistic_dim, hidden_dim, num_classes, ffnn_hidden_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define the optimizer with learning rate

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Data Loader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        # Splitting the input tensor into three feature sets
        acoustic_features = inputs[:, :acoustic_end]
        paralinguistic_features = inputs[:, acoustic_end:paralinguistic_end]
        linguistic_features = inputs[:, paralinguistic_end:]

        optimizer.zero_grad()
        outputs = model(acoustic_features, paralinguistic_features, linguistic_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}')

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Splitting the input tensor into three feature sets
            acoustic_features = inputs[:, :acoustic_end]
            paralinguistic_features = inputs[:, acoustic_end:paralinguistic_end]
            linguistic_features = inputs[:, paralinguistic_end:]

            outputs = model(acoustic_features, paralinguistic_features, linguistic_features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy on Test Data: {accuracy:.2f}%')


Batch input shape: torch.Size([32, 1800])
Batch labels shape: torch.Size([32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8 and 6x128)

In [ ]:
# Load the data
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')

# Print the shape of each dataframe
print("Acoustic Data Shape:", acoustic_data.shape)
print("Paralinguistic Data Shape:", paralinguistic_data.shape)
print("Linguistic Data Shape:", linguistic_data.shape)


Acoustic Data Shape: (213, 770)
Paralinguistic Data Shape: (213, 1026)
Linguistic Data Shape: (213, 8)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Load the data
acoustic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/ACOUSTIC.csv')
paralinguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Paralinguistic.csv')
linguistic_data = pd.read_csv('/content/drive/MyDrive/Autistic/Autistic_Features/Linguistic_features.csv')

# Concatinate datasets on 'name'
merged_data = acoustic_data.merge(paralinguistic_data, on='name').merge(linguistic_data, on='name')

# Preprocess data
X = merged_data.drop(columns=['name', 'Class']).values
y = merged_data['Class'].values

# Standardize and normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transformer Model
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, num_layers, num_classes, ffnn_hidden_dim, dropout_rate):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout_rate, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Feedforward layer
        self.ffnn = nn.Linear(hidden_dim, ffnn_hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

        # Final classification layer
        self.fc = nn.Linear(ffnn_hidden_dim, num_classes)

    def forward(self, src):
        src = self.embedding(src)
        out = self.transformer_encoder(src)
        out = self.ffnn(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc(out)
        return out

# Model Configuration
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_layers = 2
num_classes = len(set(y_train))
ffnn_hidden_dim = 256
dropout_rate = 0.1

model = TransformerEncoder(input_dim, num_heads, hidden_dim, num_layers, num_classes, ffnn_hidden_dim, dropout_rate)

# Loss, Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Data Loaders
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



In [8]:
# Function to Evaluate the Model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return 100 * correct / total

# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        total_loss += loss.item()

    scheduler.step()  # Adjust the learning rate
    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}')

    # Optional: Print validation accuracy every epoch
    validation_accuracy = evaluate_model(model, test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {validation_accuracy:.2f}%')

# Evaluate the model on the test set
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy on Test Data: {accuracy:.2f}%')

# Calculate and print the classification report
true_labels = y_test  # True labels from the test set
predicted_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.cpu().numpy())

classification_rep = classification_report(true_labels, predicted_labels)
print("Classification Report:\n", classification_rep)

Epoch 1/100, Average Loss: 0.2252
Epoch 1/100, Validation Accuracy: 100.00%
Epoch 2/100, Average Loss: 0.0274
Epoch 2/100, Validation Accuracy: 81.40%
Epoch 3/100, Average Loss: 0.0337
Epoch 3/100, Validation Accuracy: 97.67%
Epoch 4/100, Average Loss: 0.1026
Epoch 4/100, Validation Accuracy: 95.35%
Epoch 5/100, Average Loss: 0.1020
Epoch 5/100, Validation Accuracy: 97.67%
Epoch 6/100, Average Loss: 0.0110
Epoch 6/100, Validation Accuracy: 97.67%
Epoch 7/100, Average Loss: 0.0087
Epoch 7/100, Validation Accuracy: 97.67%
Epoch 8/100, Average Loss: 0.0021
Epoch 8/100, Validation Accuracy: 97.67%
Epoch 9/100, Average Loss: 0.0006
Epoch 9/100, Validation Accuracy: 97.67%
Epoch 10/100, Average Loss: 0.0003
Epoch 10/100, Validation Accuracy: 97.67%
Epoch 11/100, Average Loss: 0.0086
Epoch 11/100, Validation Accuracy: 97.67%
Epoch 12/100, Average Loss: 0.0002
Epoch 12/100, Validation Accuracy: 97.67%
Epoch 13/100, Average Loss: 0.0002
Epoch 13/100, Validation Accuracy: 97.67%
Epoch 14/100, Av